# Create elementary objects

In [1]:
from melusine.nlp_tools.normalizer import Normalizer
from melusine.nlp_tools.tokenizer import RegexTokenizer
from melusine.nlp_tools.phraser import Phraser
from melusine.nlp_tools.text_processor import TextProcessor, make_tokenizer
from melusine.nlp_tools.text_flagger import DeterministicTextFlagger
from melusine.nlp_tools.token_flagger import FlashtextTokenFlagger
from melusine.nlp_tools.lemmatizer import DummyLemmatizer
from melusine.core.pipeline import MelusinePipeline
from melusine.nlp_tools.embedding import Embedding

from melusine import load_email_data

Using pandas backend for Data transformations


In [2]:
n = Normalizer(form="NFKD", lowercase=True)
t = RegexTokenizer(tokenizer_regex=r"\w+(?:[\?\-\"_]\w+)*", stopwords=["le", "les"])
textf = DeterministicTextFlagger(text_flags = {r"\d{10}": "flag_phone"})
tokenf = FlashtextTokenFlagger(token_flags = {"flag_name": ["joe", "bob"]})
d = DummyLemmatizer()

In [3]:
e = Embedding(min_count=2)

# Text Processor object

In [4]:
pp = TextProcessor(
        normalizer=n,
        text_flagger=textf,
        tokenizer=t,
        token_flagger=tokenf,
        lemmatizer=d,
)

In [5]:
pp.process("Appelle bob, sont numéro est le 0611111111 ! Il a les billets")

['appelle',
 'flag_name',
 'sont',
 'numero',
 'est',
 'flag_phone',
 'il',
 'a',
 'billet']

In [6]:
# pp.save("my_text_processor")

In [7]:
# ppp = TextProcessor.load("my_text_processor")

In [8]:
# ppp.process("les écureuils sont présents !!!")

# Melusine Pipeline

In [9]:
df = load_email_data(type="full")
df["text"] = df["body"]
df.head(2)

,body,header,date,from,to,attachment,sexe,age,label,is_begin_by_transfer,...,min__44,min__45,min__49,min__52,min__54,min__56,min__58,attachment_type__0,attachment_type__1,text
0,\n \n \n \n Bonjour \n Je suis client chez...,Devis habitation,2018-05-24 11:36:00,Dupont <monsieurdupont@extensiona.com>,conseiller@Societeimaginaire.fr,[],F,35,habitation,True,...,0,0,0,0,0,0,0,0,1,\n \n \n \n Bonjour \n Je suis client chez...
1,"\n \n \n \n Bonsoir madame, \n \n Je vous...",Immatriculation voiture,2018-05-24 19:37:00,Dupont <monsieurdupont@extensiona.com>,conseiller@Societeimaginaire.fr,"[""pj.pdf""]",M,32,vehicule,True,...,0,0,0,0,0,0,0,1,0,"\n \n \n \n Bonsoir madame, \n \n Je vous..."


## Add a custom SKlearn transformer to the pipeline

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

class Stupid(BaseEstimator, TransformerMixin):
    def fit(self, df, y=None):
        return self

    def transform(self, df):
        df["stupid"] = True
        return df    

In [11]:
stupid = Stupid()

## Assemble Pipeline

In [12]:
textf2 = DeterministicTextFlagger(text_flags = {r"je": "JE"})
gensim_phraser = Phraser(threshold=2, min_count=2, input_columns=["tokens"], output_columns=["tokens"])

m_pipe = MelusinePipeline([
        ("normalizer", n),
        ("text_flagger", textf),
        ("text_flagger2", textf2),
        ("tokenizer", t),
        ("lemmatizer", d),
        ("stupid", stupid),
        ("gensim_phraser", gensim_phraser),
        ("token_flagger", tokenf),
        ("w2v", e),
],
    verbose=True
)

## Execute pipeline

In [13]:
df = m_pipe.fit_transform(df)

[Pipeline] ........ (step 1 of 9) Processing normalizer, total=   0.0s
[Pipeline] ...... (step 2 of 9) Processing text_flagger, total=   0.0s
[Pipeline] ..... (step 3 of 9) Processing text_flagger2, total=   0.0s
[Pipeline] ......... (step 4 of 9) Processing tokenizer, total=   0.0s
[Pipeline] ........ (step 5 of 9) Processing lemmatizer, total=   0.0s
[Pipeline] ............ (step 6 of 9) Processing stupid, total=   0.0s
[Pipeline] .... (step 7 of 9) Processing gensim_phraser, total=   0.0s
[Pipeline] ..... (step 8 of 9) Processing token_flagger, total=   0.0s
[Pipeline] ............... (step 9 of 9) Processing w2v, total=   0.0s


In [14]:
print("stupid" in df.columns)
df.iloc[0]["tokens"]

True


['bonjour_JE',
 'sui',
 'client',
 'chez',
 'vou',
 'pouvez',
 'vou',
 'm',
 'etablir',
 'un_devi',
 'pour',
 'mon',
 'fil',
 'qui',
 'souhaite',
 'louer',
 'lappartement',
 'suivant',
 '25',
 'rue',
 'du',
 'rueimaginaire',
 '77000',
 'merci',
 'envoye_de',
 'mon_iphone']

In [15]:
m_pipe.save("my_pipeline")

In [16]:
m_pipe_reloaded = MelusinePipeline.load("my_pipeline")

In [17]:
m_pipe_reloaded

MelusinePipeline(steps=[('normalizer',
                         Normalizer(input_columns=['text'],
                                    output_columns=['text'])),
                        ('text_flagger',
                         DeterministicTextFlagger(input_columns=['text'],
                                                  output_columns=['text'],
                                                  text_flags={'\\d{10}': 'flag_phone'})),
                        ('text_flagger2',
                         DeterministicTextFlagger(input_columns=['text'],
                                                  output_columns=['text'],
                                                  text_flags={'je': 'JE'})),
                        ('tokenizer'...
                        ('stupid', Stupid()),
                        ('gensim_phraser',
                         Phraser(input_columns=['tokens'],
                                 output_columns=['tokens'])),
                        ('token_flagger'

In [18]:
df = load_email_data(type="full")
df["text"] = df["body"]
df = m_pipe_reloaded.transform(df)
print("stupid" in df.columns)
df.iloc[0]["tokens"]

True


['bonjour_JE',
 'sui',
 'client',
 'chez',
 'vou',
 'pouvez',
 'vou',
 'm',
 'etablir',
 'un_devi',
 'pour',
 'mon',
 'fil',
 'qui',
 'souhaite',
 'louer',
 'lappartement',
 'suivant',
 '25',
 'rue',
 'du',
 'rueimaginaire',
 '77000',
 'merci',
 'envoye_de',
 'mon_iphone']

In [19]:
import numpy as np
m_pipe_reloaded.named_steps["w2v"].embeddings_["date"], np.ndarray

(array([ 9.0266630e-04,  6.4310022e-03, -6.7403787e-03,  7.1425955e-03,
         1.0331177e-02,  1.9011955e-03,  3.9349636e-03,  6.4286543e-03,
        -2.9777391e-03, -8.4869433e-03,  8.1091013e-04, -5.0247193e-04,
        -7.2138435e-03,  2.3347558e-03,  4.8175771e-03, -8.1246467e-03,
         2.5347041e-03,  6.5802303e-03, -8.8761132e-03,  1.1852063e-03,
        -8.1245080e-03, -7.6027690e-03,  7.6509570e-03, -7.9380795e-03,
        -1.8591663e-03, -7.8568542e-03, -8.5925832e-03, -8.1224414e-03,
        -1.2418941e-03,  2.2142131e-03, -6.9472692e-03, -7.7581573e-03,
        -2.5859969e-03, -2.4417292e-03, -6.8943468e-03, -2.9456376e-03,
         3.4222791e-03,  1.8526295e-03,  5.4572974e-03,  2.1080021e-03,
        -5.4242183e-04,  1.3953992e-03,  8.9178765e-03,  2.0206973e-03,
         3.1961922e-03,  4.3547744e-04, -9.5608933e-03,  4.6774661e-03,
         5.7900925e-03,  4.0709460e-04,  1.0159505e-02, -6.6816816e-03,
         3.5485327e-03, -7.7318372e-03, -9.7941346e-03,  2.79994

In [20]:
m_pipe_reloaded.named_steps["gensim_phraser"].phraser_.save("my_pipeline/phraser_obj")

# Pipeline Visualization

In [21]:
from sklearn import set_config

set_config(display='diagram')
m_pipe_reloaded

MelusinePipeline(steps=[('normalizer',
                         Normalizer(input_columns=['text'],
                                    output_columns=['text'])),
                        ('text_flagger',
                         DeterministicTextFlagger(input_columns=['text'],
                                                  output_columns=['text'],
                                                  text_flags={'\\d{10}': 'flag_phone'})),
                        ('text_flagger2',
                         DeterministicTextFlagger(input_columns=['text'],
                                                  output_columns=['text'],
                                                  text_flags={'je': 'JE'})),
                        ('tokenizer'...
                        ('stupid', Stupid()),
                        ('gensim_phraser',
                         Phraser(input_columns=['tokens'],
                                 output_columns=['tokens'])),
                        ('token_flagger',
                         FlashtextTokenFlagger(flashtext_separators=['-', '_',
                                                                     '/'],
                                               input_columns=['tokens'],
                                               output_columns=['tokens'],
                                               token_flags={'flag_name': ['joe',
                                                                          'bob']})),
                        ('w2v',
                         Embedding(input_columns=['tokens'],
                                   output_columns=['tokens']))],
                 verbose=True)

# Simplified Pipeline creation

In [22]:
easy_pipe = make_tokenizer(
        form = "NFKD",
        lowercase = True,
        tokenizer_regex = r"\w+(?:[\?\-\"_]\w+)*",
        stopwords = ["le", "les"],
        text_flags = {r"\d{10}": "flag_phone"},
        token_flags = {"flag_name": ["joe", "bob"]},  
)

In [23]:
easy_pipe

MelusinePipeline(memory=False,
                 steps=[('normalizer',
                         Normalizer(input_columns=('text',),
                                    output_columns=('text',))),
                        ('text_flagger',
                         DeterministicTextFlagger(input_columns=('text',),
                                                  output_columns=('text',),
                                                  text_flags={'\\d{10}': 'flag_phone'})),
                        ('tokenizer',
                         RegexTokenizer(input_columns=('text',),
                                        output_columns=('tokens',),
                                        stopwords={'le', 'les'})),
                        ('token_flagger',
                         FlashtextTokenFlagger(input_columns=('tokens',),
                                               output_columns=('tokens',),
                                               token_flags={'flag_name': ['joe',
                                                                          'bob']}))],
                 verbose=False)

In [24]:
df = load_email_data(type="full")
df["text"] = df["body"]
df = easy_pipe.transform(df)
df.iloc[0]["tokens"]

['bonjour',
 'je',
 'suis',
 'client',
 'chez',
 'vous',
 'pouvez',
 'vous',
 'm',
 'etablir',
 'un',
 'devis',
 'pour',
 'mon',
 'fils',
 'qui',
 'souhaite',
 'louer',
 'lappartement',
 'suivant',
 '25',
 'rue',
 'du',
 'rueimaginaire',
 '77000',
 'merci',
 'envoye',
 'de',
 'mon',
 'iphone']

In [25]:
easy_pipe.save("my_easy_pipeline")

In [26]:
easy_pipe_reloaded = MelusinePipeline.load("my_easy_pipeline")

In [27]:
df = load_email_data(type="full")
df["text"] = df["body"]
df = easy_pipe_reloaded.transform(df)
df.iloc[0]["tokens"]

['bonjour',
 'je',
 'suis',
 'client',
 'chez',
 'vous',
 'pouvez',
 'vous',
 'm',
 'etablir',
 'un',
 'devis',
 'pour',
 'mon',
 'fils',
 'qui',
 'souhaite',
 'louer',
 'lappartement',
 'suivant',
 '25',
 'rue',
 'du',
 'rueimaginaire',
 '77000',
 'merci',
 'envoye',
 'de',
 'mon',
 'iphone']

# Pipeline composition

In [28]:
p1 = MelusinePipeline([
        ("normalizer", n),
        ("text_flagger", textf),
        ("text_flagger2", textf2),
        ("tokenizer", t),
],
    verbose=True
)
p2 = MelusinePipeline([
        ("lemmatizer", d),
        ("stupid", stupid),
        ("gensim_phraser", gensim_phraser),
        ("token_flagger", tokenf),
],
    verbose=True
)

In [29]:
p3 = MelusinePipeline([("text_pipe", p1), ("token_pipe", p2)])

In [30]:
p3.transform(df).head(2)

,body,header,date,from,to,attachment,sexe,age,label,is_begin_by_transfer,...,min__49,min__52,min__54,min__56,min__58,attachment_type__0,attachment_type__1,text,tokens,stupid
0,\n \n \n \n Bonjour \n Je suis client chez...,Devis habitation,2018-05-24 11:36:00,Dupont <monsieurdupont@extensiona.com>,conseiller@Societeimaginaire.fr,[],F,35,habitation,True,...,0,0,0,0,0,0,1,\n \n \n \n bonjour \n JE suis client chez...,"[bonjour_JE, sui, client, chez, vou, pouvez, v...",True
1,"\n \n \n \n Bonsoir madame, \n \n Je vous...",Immatriculation voiture,2018-05-24 19:37:00,Dupont <monsieurdupont@extensiona.com>,conseiller@Societeimaginaire.fr,"[""pj.pdf""]",M,32,vehicule,True,...,0,0,0,0,0,1,0,"\n \n \n \n bonsoir madame, \n \n JE vous...","[bonsoir, madame_JE, vou, informe, que, la_nou...",True


In [31]:
p3

MelusinePipeline(steps=[('text_pipe',
                         MelusinePipeline(steps=[('normalizer',
                                                  Normalizer(input_columns=('text',),
                                                             output_columns=('text',))),
                                                 ('text_flagger',
                                                  DeterministicTextFlagger(input_columns=('text',),
                                                                           output_columns=('text',),
                                                                           text_flags={'\\d{10}': 'flag_phone'})),
                                                 ('text_flagger2',
                                                  DeterministicTextFlagger(input_columns=('text',),
                                                                           output_columns=('te...
                         MelusinePipeline(steps=[('lemmatizer',
                                                  DummyLemmatizer(input_columns=('tokens',),
                                                                  output_columns=('tokens',))),
                                                 ('stupid', Stupid()),
                                                 ('gensim_phraser',
                                                  Phraser(input_columns=['tokens'],
                                                          output_columns=['tokens'])),
                                                 ('token_flagger',
                                                  FlashtextTokenFlagger(input_columns=('tokens',),
                                                                        output_columns=('tokens',),
                                                                        token_flags={'flag_name': ['joe',
                                                                                                   'bob']}))],
                                          verbose=True))])

In [32]:
p3.save("pipeline_compo")

In [33]:
p3_reloaded = MelusinePipeline.load("pipeline_compo")
p3_reloaded

MelusinePipeline(steps=[('text_pipe',
                         MelusinePipeline(steps=[('normalizer',
                                                  Normalizer(input_columns=['text'],
                                                             output_columns=['text'])),
                                                 ('text_flagger',
                                                  DeterministicTextFlagger(input_columns=['text'],
                                                                           output_columns=['text'],
                                                                           text_flags={'\\d{10}': 'flag_phone'})),
                                                 ('text_flagger2',
                                                  DeterministicTextFlagger(input_columns=['text'],
                                                                           output_columns=['text'],...
                                                  DummyLemmatizer(input_columns=('tokens',),
                                                                  output_columns=('tokens',))),
                                                 ('stupid', Stupid()),
                                                 ('gensim_phraser',
                                                  Phraser(input_columns=['tokens'],
                                                          output_columns=['tokens'])),
                                                 ('token_flagger',
                                                  FlashtextTokenFlagger(flashtext_separators=['-',
                                                                                              '_',
                                                                                              '/'],
                                                                        input_columns=['tokens'],
                                                                        output_columns=['tokens'],
                                                                        token_flags={'flag_name': ['joe',
                                                                                                   'bob']}))],
                                          verbose=True))])

In [34]:
df = load_email_data(type="full")
df["text"] = df["body"]
df = m_pipe_reloaded.transform(df)
print("stupid" in df.columns)
df.iloc[0]["tokens"]

True


['bonjour_JE',
 'sui',
 'client',
 'chez',
 'vou',
 'pouvez',
 'vou',
 'm',
 'etablir',
 'un_devi',
 'pour',
 'mon',
 'fil',
 'qui',
 'souhaite',
 'louer',
 'lappartement',
 'suivant',
 '25',
 'rue',
 'du',
 'rueimaginaire',
 '77000',
 'merci',
 'envoye_de',
 'mon_iphone']

# Change Pipeline Execution Backend

In [35]:
from melusine.backend.melusine_backend import use, backend

In [36]:
use("dict")

Using dict backend for Data transformations


In [37]:
backend._backend

In [38]:
ddd = df.iloc[0].to_dict()

In [39]:
type(m_pipe.transform(ddd))

dict

In [40]:
m_pipe.transform(ddd)

{'body': ' \n  \n  \n  \n Bonjour \n Je suis client chez vous \n Pouvez vous m établir un devis pour mon fils qui souhaite \n louer l’appartement suivant : \n 25 rue du rueimaginaire 77000 \n Merci \n Envoyé de mon iPhone',
 'header': 'Devis habitation',
 'date': '2018-05-24 11:36:00',
 'from': 'Dupont <monsieurdupont@extensiona.com>',
 'to': 'conseiller@Societeimaginaire.fr',
 'attachment': '[]',
 'sexe': 'F',
 'age': 35,
 'label': 'habitation',
 'is_begin_by_transfer': True,
 'is_answer': False,
 'is_transfer': False,
 'structured_historic': "[{'text': ' \\n  \\n  \\n  \\n Bonjour \\n Je suis client chez vous \\n Pouvez vous m établir un devis pour mon fils qui souhaite \\n louer l’appartement suivant : \\n 25 rue du rueimaginaire 77000 \\n Merci \\n Envoyé de mon iPhone', 'meta': ''}]",
 'structured_body': "[{'meta': {'date': None, 'from': None, 'to': None}, 'structured_text': {'header': None, 'text': [{'part': 'Bonjour', 'tags': 'HELLO'}, {'part': 'Je suis client chez vous Pouvez v

# Help users understand the Framework

In [41]:
from melusine.core.melusine_transformer import MelusineTransformer

In [42]:
class NoFilename(MelusineTransformer):

    def __init__(self, input_columns=("text",), output_columns=("text",)):
        super().__init__(input_columns, output_columns)
        
    def load(self):
        pass
    
    def save(self):
        pass    
        
nofilename = NoFilename()

TypeError: Can't instantiate abstract class NoFilename with abstract methods FILENAME

In [ ]:
class NoFunc(MelusineTransformer):
    FILENAME = "nofunc"
    def __init__(self, input_columns=("text",), output_columns=("text",)):
        super().__init__(input_columns, output_columns)
        
    def load(self):
        pass
    
    def save(self):
        pass    
        
nofunc = NoFunc()

In [ ]:
print(nofunc.transform(ddd))

# Regex definition

In [ ]:
# === Info importante ===

# Les sauts de lignes sont remplacés par le pattern " ; " à la réception de la requête
# C'est clairement une transformation indésirable aujourd'hui 
# mais elle est encrée dans les codes et il faudrait prendre quelques jours pour modifier ça et étudier les impacts !


# === Start pattern ===
# On cherche un début de ligne ou un ";"
start_pattern = r"""(?:^|;)"""

# === Symboles de début de ligne ===
# Les emails avec des multiples retours à la ligne génèrent des paterns de " ; ; ; ; ; ; "
# Certains messages transférés / réponses ont des symboles en début de lignes (> et/ou |)
# Ex:
# Merci
# > De foo@maif.fr A bar@gmail.com
# > Voici le document
# On ignore tous ces symboles
ignore_characters = """(?:[>| ;]*)"""

# === Keywords de transition ===
# Certains mot-clés présents dans les réponses et emails transférés sont utilisés pour la segmentation
# Ces mot clés sont suivis du symbole ":"
# Ex:
# De : XX A : XX Sujet : Blah Blah
meta_transition_words = """(?:\\b(?:[Ee]nvoy[ée](?: par)?|[Dd]e|[Oo]bjet|[Cc]c|Date|[AÀàa]|[Dd]estinataire|[Ss]ent|[Tt]o|[Ss]ub?jec?t|[Ff]rom|[Cc]opie [àa])\\b\\s{,4}:)"""

# === Méta-données ===
# Les mots-clés sont suivi de champs de textes libre qu'il faut identifier
# On accepte un retour à la ligne à la suite du mot clé ("\s{,4};\s{,4}")
# On limite le champs de texte libre à 150 charactères (car un .* est très couteux en temps de calcul)
# Le champ de texte libre s'arrête lorsqu'on détecte un ";" (retour à la ligne)
meta_content_pattern = r"""(?:\\s{,4};\\s{,4}[^;]{,100}[;|]\\s{,4})"""


# === Meta data pattern ===
meta_pattern = fr"(?:{ignore_characters}{meta_transition_words}{meta_content_pattern})"

# === Full pattern ===
# On cherche un start pattern suivi de répétitions de meta_pattern
regex = fr"""({start_pattern}{meta_pattern}+)"""

In [ ]:
print(regex)

In [ ]:
print("""(?:(?:^|;)(?:(?:[>| ;]*(?:Envoy[ée]|De|Objet|Cc|Envoy[ée] par|Date|A|À|Destinataire|Sent|To|Subject|Sujet|From|Copie [àa])\s{,4}:\s{,4};?\s{,4}[^;]{,100}[;|]\s{,4}))+)""")

In [ ]:
print("""| |\n| |\n| |-------- Message transféré --------\n| |\n| |Sujet :\n| | [INTERNET] Dossier F210306856A -\n| | Date :| | Mon, 13 Sep 2021 13:45:39 +0200 | | De :| | gestionsinistre@maif.fr | | Pour :| | anomalies-vol-siv@interieur.gouv.fr | | | |Bonjour, |""")

In [ ]:
import json
json.dumps(regex)

In [ ]:
regex

In [ ]:
x = "\\s"
y = r"\s"

In [ ]:
json.dumps(x)

In [ ]:
json.dumps(y)